In [192]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from datetime import datetime
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

almaURL = 'https://unl.alma.exlibrisgroup.com/mng/login?auth=SAML'
webdriverURL = '/Users/danstara/Downloads/chromedriver'
inputDataPath = '/Users/danstara/Downloads/NUIDsToDo.csv'

#auth information
authUsername = '_'
authPassword = '_'

#html IDs
expirationDateTextId = 'SPAN_SELENIUM_ID_userList_ROW_0_COL_expireDateDummy'
almaSearchInputId = 'ALMA_MENU_TOP_NAV_Search_Text'

In [194]:
def check_alma_for_user(userID):
    searchElement = browser.find_element("id", almaSearchInputId) #it rerenders, so finding it each time reduces bugs
    searchElement.clear()
    searchElement = browser.find_element("id", almaSearchInputId)
    searchElement.send_keys(userID)
    searchElement = browser.find_element("id", almaSearchInputId)
    searchElement.send_keys(Keys.ENTER)
    time.sleep(3)
    searchElement = browser.find_element("id", almaSearchInputId)
    searchElement.clear()
    
    if "No records were found." in browser.page_source:
        return False
    else:
        return True
    

def navigateToPage():
    browser.get(almaURL)
    time.sleep(3)
    usernameElement = browser.find_element("id", "username")
    usernameElement.send_keys(authUsername)
    passwordElement = browser.find_element("id", "password")
    passwordElement.send_keys(authPassword)
    usernameElement.send_keys(Keys.ENTER)
    time.sleep(24)
    

def dateIsBeforeToday(dateString):
    past = datetime.strptime(dateString, "%m/%d/%Y")
    present = datetime.now()
    return past.date() < present.date()


In [1]:
s=Service(webdriverURL)
browser = webdriver.Chrome(service=s)
navigateToPage()
        
inputNUIDs = pd.read_csv(inputDataPath)

generated = []
for num in inputNUIDs["NUID"]:
    if check_alma_for_user(num):
        expiredDateElementText = browser.find_element("id", expirationDateTextId).text
        if(dateIsBeforeToday(expiredDateElementText)):
            generated.append("EXPIRED")
        else:
            generated.append("YES")
        
    else:
        generated.append("NO")
    
# inputNUIDs["Generated"] = generated
print("done")
print(generated)


KeyboardInterrupt



In [198]:
# output the data
shortInputNUIDs = inputNUIDs.head(len(generated))
shortInputNUIDs["Generated"] = generated
shortInputNUIDs

In [220]:
shortInputNUIDs.to_csv('/Users/danstara/Downloads/output4.csv')